In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
# Standard library imports
import sys
import os
import json

# Third-party library imports
import pandas as pd
import numpy as np
import joblib
import shap

# Set pandas display options
pd.set_option("display.max_rows", 200)

# Define file location
PY_FILE_LOC = os.getcwd()

# Project-specific utility imports
from utils.japan_admin_data import (
    prefecture_dict_jp_to_en,
    japanadmin_muni_all_jp_to_en,
)

from utils.nbutils_load_data import load_and_process_data, get_scale_param
from utils.nbutils_cluster_stats import get_pref_muni_isin, apply_scale


In [3]:
# load the base data
ftr_pref_muni = ["pref", "muni"]
vars_iv = ["demand", "land_avail", "taxable_income", "LV", "SPR", "pv_out"]

folder = os.path.join(PY_FILE_LOC, "data")
pv_params, df = load_and_process_data(folder)

outlier_summary = pd.read_csv(os.path.join(PY_FILE_LOC, "data", "pv_growth_outlier.csv"))
outlier_index = outlier_summary[ftr_pref_muni].apply(tuple, axis=1).to_list()

pv_param_no_outliers = pv_params[
    ~(pv_params[ftr_pref_muni].apply(tuple, axis=1).isin(outlier_index))
]
pv_param_outliers = pv_params[
    (pv_params[ftr_pref_muni].apply(tuple, axis=1).isin(outlier_index))
]

scale_param = get_scale_param()

In [4]:
scale_param

,unit,scaler,unit_scaled,mean_sno,std_sno
demand,MWh,1000.00,GWh,0,0
land_avail,ha,1.00,ha,0,0
taxable_income,JPY,1000000.00,M JPY,0,0
LV,M JPY,1000.00,B JPY,0,0
pv_out,kWh/kW/year,1.00,kWh/kW/year,0,0
SPR,unit,0.01,%,2,2


In [5]:
# outlier utils
def is_float_between(value, range_tuple):
    lower_bound, upper_bound = range_tuple
    return lower_bound < value < upper_bound

def get_closest_cluster(value, cluster_pv_range):
	vals = []
	for k, values in cluster_pv_range.items():
		for v in values:
			vals.append([v, k])
	vals = pd.DataFrame(vals, columns=['values', 'cluster']).sort_values('values').reset_index(drop=True)
	if value > vals['values'].max():
		return "-"
	vals['residue'] = abs(vals['values'] - value)
	return int(vals.sort_values('residue').iloc[0]['cluster'])


def get_potential_cluster(value, cluster_pv_range):
    potential_list = []
    for cluster, pv_range in cluster_pv_range.items():
        if is_float_between(value, pv_range):
            potential_list.append(str(cluster))
    if len(potential_list) == 0:
        return get_closest_cluster(value, cluster_pv_range)
    return ",".join(potential_list)


def get_cluster_pv_range(df, cluster_index, scale_param, vars_dv):
    pv_range_val = dict()
    scale_dict = scale_param["scaler"].to_dict()

    for cluster_no, cluster_indx in cluster_index.items():
        df_temp = get_pref_muni_isin(df, cluster_indx)
        df_temp = apply_scale(df_temp, scale_dict)
        df_temp = df_temp[scale_dict.keys()]
        pv_range_val[cluster_no] = (
            float(df_temp[vars_dv].min().round(4)),
            float(df_temp[vars_dv].max().round(4)),
        )
    return pv_range_val

In [6]:
# load the models and intermediate files
model_rfr = dict()
model_rfc = dict()
shap_score_summary = dict()
cluster_pv_range = dict()

year = 2023
df_year = df[df["year"] == year]

for vars_type in ["PV_R", "PV_S"]:
    model_rfr[vars_type] = joblib.load(
        os.path.join(PY_FILE_LOC, "data", f"model_RFR_{vars_type}.joblib")
    )
    model_rfc[f"{vars_type}_{year}"] = joblib.load(
        os.path.join(PY_FILE_LOC, "data", f"model_RFC_{vars_type}_2023.joblib")
    )
    shap_score_summary[vars_type] = pd.read_csv(
        os.path.join(PY_FILE_LOC, "data", f"shap_values_summary_{vars_type}.csv")
    )
    cluster_index = dict()
    for kmean_no, df_g in shap_score_summary[vars_type].groupby("kmeans_cluster"):
        cluster_index[kmean_no] = df_g[ftr_pref_muni].apply(tuple, axis=1).to_list()

    scale_param_temp = scale_param.copy()
    scale_param_temp.loc[vars_type] = {
		"unit": "%",
		"scaler": 1,
		"unit_scaled": "%",
		"mean_sno": 2,
		"std_sno": 2,
	}

    cluster_pv_range[vars_type] = get_cluster_pv_range(
		df_year, cluster_index, scale_param_temp, vars_type
	)

cluster_pv_range

{'PV_R': {0: (0.5033, 0.9216),
  1: (0.2478, 0.622),
  2: (0.0718, 0.3427),
  3: (0.0405, 0.2253),
  4: (0.001, 0.2169),
  5: (0.0221, 0.1533),
  6: (0.0028, 0.1392),
  7: (0.0, 0.0808)},
 'PV_S': {0: (0.3135, 0.708),
  1: (0.1389, 0.4664),
  2: (0.0555, 0.3469),
  3: (0.0657, 0.3393),
  4: (0.0018, 0.1554),
  5: (0.0005, 0.0954),
  6: (0.0, 0.1191),
  7: (0.0, 0.1382)}}

In [7]:
# ============================================================================ #
# Predict Outliers PV_M 2023
# ============================================================================ #
selected_outliers = dict()
outlier_ratio_cap = {'PV_R': 1.1, 'PV_S':1.1}
for vars_dv in ['PV_S', 'PV_R']:
	cols = []
	year = 2023

	var_dv_year = f"{vars_dv}_{year}"
	cols.extend(ftr_pref_muni)
	cols.extend(
		["demand", "land_avail", "taxable_income", f"LV_{year}", f"SPR_{year}", "pv_out"]
	)
	cols.extend([var_dv_year])
	df_outliers = pv_param_outliers[cols].copy()
	rename_cols = {
		c: c.replace(f"_{year}", "") for c in df_outliers.filter(regex=f"{year}$").columns
	}

	df_outliers = df_outliers.rename(columns=rename_cols)
	df_outliers[vars_dv] = (
		100 * df_outliers[vars_dv] / pv_param_no_outliers[var_dv_year].sum()
	)
	df_outliers.sort_values(vars_dv, ascending=False)

	# load the shap
	explainer = shap.TreeExplainer(model_rfr[vars_dv])
	X_outliers = df_outliers[vars_iv]
	shap_values_outliers_df = explainer.shap_values(X_outliers)
	shap_values_outliers_df = pd.DataFrame(shap_values_outliers_df, columns=vars_iv)

	shap_values_outliers_df.columns = [
		f"{c}_score" for c in shap_values_outliers_df.columns
	]
	shap_values_outliers_df[f"{vars_dv}_shap"] = (
		shap_values_outliers_df.sum(axis=1) + explainer.expected_value
	)
	shap_values_outliers_df = pd.concat(
		[df_outliers.reset_index(drop=True), shap_values_outliers_df], axis=1
	)
	shap_values_outliers_df.sort_values(vars_dv, ascending=False).reset_index(drop=True)


	shap_values_outliers_df["inc_ratio"] = (
		shap_values_outliers_df[vars_dv] / shap_values_outliers_df[f"{vars_dv}_shap"]
	)
	shap_values_outliers_df["pref_en"] = shap_values_outliers_df["pref"].map(
		prefecture_dict_jp_to_en
	)
	shap_values_outliers_df["muni_en"] = shap_values_outliers_df["muni"].map(
		japanadmin_muni_all_jp_to_en
	)
	shap_values_outliers_df.head(5)

	X_outliers_clustering = shap_values_outliers_df[vars_iv]
	
	# using the parameters of the city, the model predicts what cluster it should be. 
	# since the these are outliers. The range will fall outside the prediction of the model
	shap_values_outliers_df["cluster_p"] = model_rfc[var_dv_year].predict(X_outliers_clustering)
	selected_outliers_temp = (
		# shap_values_outliers_df[shap_values_outliers_df["inc_ratio"] > outlier_ratio_cap[vars_dv]]
		shap_values_outliers_df
		.sort_values("inc_ratio", ascending=False)
		.reset_index(drop=True)
	)

	# uses the actual value and the cluster ranges and assigns where the actual value should fall
	selected_outliers_temp['cluster_a'] = selected_outliers_temp[vars_dv].apply(get_potential_cluster, args=(cluster_pv_range[vars_dv], ))

	# scale the variables
	scale_dict = scale_param["scaler"].to_dict()
	selected_outliers_temp = apply_scale(selected_outliers_temp, scale_dict)

	selected_outliers[vars_dv] = selected_outliers_temp.copy()


In [8]:
selected_outliers_temp

,pref,muni,demand,land_avail,taxable_income,LV,SPR,pv_out,PV_R,demand_score,...,taxable_income_score,LV_score,SPR_score,pv_out_score,PV_R_shap,inc_ratio,pref_en,muni_en,cluster_p,cluster_a
0,鹿児島県,鹿屋市,591.05000,9065.324700,119.774811,9.196,15.8550,1270.350996,0.190728,-0.017206,...,0.037447,-0.005501,0.006899,-0.004682,0.077162,2.471788,Kagoshima,Kanoya-shi,6,"2,3,4"
1,愛知県,名古屋市,13193.57400,15344.179300,4901.686190,164.971,11.8176,1345.327967,1.408996,0.594582,...,0.055080,0.033126,0.006873,-0.021857,0.783114,1.799222,Aichi,Nagoya-shi,0,-
2,静岡県,浜松市,4875.02700,26724.051600,1346.150131,57.433,11.8176,1350.125258,1.267971,0.517996,...,0.071118,0.017795,0.001907,-0.008871,0.714349,1.775003,Shizuoka,Hamamatsu-shi,0,-
3,佐賀県,佐賀市,1349.48300,11083.835700,339.148618,32.399,15.8550,1232.951253,0.408977,0.109106,...,0.045707,0.001178,0.003147,-0.000526,0.235787,1.734522,Saga,Saga-shi,2,1
4,宮崎県,都城市,1130.13500,11346.458300,192.517371,17.543,15.8550,1289.189536,0.372737,0.088893,...,0.049145,0.010415,0.004185,-0.004872,0.225923,1.649840,Miyazaki,Miyakonojo-shi,2,1
5,愛知県,一宮市,1582.48500,4256.452000,625.426822,84.082,11.8176,1334.737280,0.545455,0.150611,...,0.096319,0.001769,0.003129,0.004871,0.344945,1.581279,Aichi,Ichinomiya-shi,2,"0,1"
6,群馬県,伊勢崎市,1703.31300,5206.815800,316.847057,36.709,9.2304,1223.507472,0.378055,0.130022,...,0.026802,0.004756,0.002759,-0.005170,0.242355,1.559923,Gunma,Isesaki-shi,2,1
7,群馬県,太田市,2184.17500,5632.729300,352.793805,35.385,9.2304,1223.507472,0.451776,0.176058,...,0.021000,0.012065,0.003356,-0.006799,0.293871,1.537327,Gunma,Ota-shi,2,1
8,宮崎県,宮崎市,2245.22600,16059.351000,555.170225,24.720,15.8550,1294.784151,0.682333,0.269545,...,0.086117,0.003979,0.004060,-0.000283,0.485850,1.404411,Miyazaki,Miyazaki-shi,1,0
9,群馬県,前橋市,2006.62200,9263.824400,541.886803,49.504,9.2304,1242.527382,0.518758,0.205770,...,0.071103,0.011068,0.003744,-0.002729,0.376105,1.379290,Gunma,Maebashi-shi,1,"0,1"


In [9]:
samples = dict()

In [10]:
ftr_pref_muni_en = ['pref_en', 'muni_en']
vars_dv = 'PV_R'
cols = []
cols.extend(ftr_pref_muni_en)
cols.extend(vars_iv)
cols.extend([vars_dv, f'{vars_dv}_shap', "inc_ratio", 'cluster_p', 'cluster_a'])
df_temp = selected_outliers[vars_dv][cols]
df_temp = df_temp[df_temp['inc_ratio']>1.3]
samples[vars_dv] = len(df_temp)
df_temp

,pref_en,muni_en,demand,land_avail,taxable_income,LV,SPR,pv_out,PV_R,PV_R_shap,inc_ratio,cluster_p,cluster_a
0,Kagoshima,Kanoya-shi,591.050,9065.3247,119.774811,9.196,15.8550,1270.350996,0.190728,0.077162,2.471788,6,"2,3,4"
1,Aichi,Nagoya-shi,13193.574,15344.1793,4901.686190,164.971,11.8176,1345.327967,1.408996,0.783114,1.799222,0,-
2,Shizuoka,Hamamatsu-shi,4875.027,26724.0516,1346.150131,57.433,11.8176,1350.125258,1.267971,0.714349,1.775003,0,-
3,Saga,Saga-shi,1349.483,11083.8357,339.148618,32.399,15.8550,1232.951253,0.408977,0.235787,1.734522,2,1
4,Miyazaki,Miyakonojo-shi,1130.135,11346.4583,192.517371,17.543,15.8550,1289.189536,0.372737,0.225923,1.649840,2,1
5,Aichi,Ichinomiya-shi,1582.485,4256.4520,625.426822,84.082,11.8176,1334.737280,0.545455,0.344945,1.581279,2,"0,1"
6,Gunma,Isesaki-shi,1703.313,5206.8158,316.847057,36.709,9.2304,1223.507472,0.378055,0.242355,1.559923,2,1
7,Gunma,Ota-shi,2184.175,5632.7293,352.793805,35.385,9.2304,1223.507472,0.451776,0.293871,1.537327,2,1
8,Miyazaki,Miyazaki-shi,2245.226,16059.3510,555.170225,24.720,15.8550,1294.784151,0.682333,0.485850,1.404411,1,0
9,Gunma,Maebashi-shi,2006.622,9263.8244,541.886803,49.504,9.2304,1242.527382,0.518758,0.376105,1.379290,1,"0,1"


In [11]:
ftr_pref_muni_en = ['pref_en', 'muni_en']
vars_dv = 'PV_S'
cols = []
cols.extend(ftr_pref_muni_en)
cols.extend(vars_iv)
cols.extend([vars_dv, f'{vars_dv}_shap', "inc_ratio", 'cluster_p', 'cluster_a'])
df_temp = selected_outliers[vars_dv][cols]
df_temp = df_temp[df_temp['inc_ratio']>1.5]
samples[vars_dv] = len(df_temp)
selected_outliers[vars_dv][cols]

,pref_en,muni_en,demand,land_avail,taxable_income,LV,SPR,pv_out,PV_S,PV_S_shap,inc_ratio,cluster_p,cluster_a
0,Okayama,Tsuyama-shi,680.78400,7289.280200,133.015107,23.375,14.2264,1297.205598,0.403329,0.084809,4.755758,4,"0,1"
1,Oita,Kitsuki-shi,154.92400,7365.332000,28.564061,11.937,15.8550,1294.901906,0.328002,0.091796,3.573143,4,"0,1,2,3"
2,Miyazaki,Miyakonojo-shi,1130.13500,11346.458300,192.517371,17.543,15.8550,1289.189536,0.650806,0.188933,3.444649,3,0
3,Gunma,Maebashi-shi,2006.62200,9263.824400,541.886803,49.504,9.2304,1242.527382,0.859678,0.259701,3.310258,0,-
4,Gunma,Isesaki-shi,1703.31300,5206.815800,316.847057,36.709,9.2304,1223.507472,0.612736,0.190604,3.214705,1,0
5,Kagoshima,Kanoya-shi,591.05000,9065.324700,119.774811,9.196,15.8550,1270.350996,0.614251,0.219348,2.800344,3,0
6,Shizuoka,Hamamatsu-shi,4875.02700,26724.051600,1346.150131,57.433,11.8176,1350.125258,1.393454,0.498970,2.792660,0,-
7,Yamanashi,Hokuto-shi,512.99000,7048.704200,60.994158,11.480,9.2304,1387.003848,0.525347,0.194935,2.694981,3,0
8,Aichi,Nagoya-shi,13193.57400,15344.179300,4901.686190,164.971,11.8176,1345.327967,0.546306,0.218218,2.503482,2,0
9,Aichi,Ichinomiya-shi,1582.48500,4256.452000,625.426822,84.082,11.8176,1334.737280,0.299124,0.156537,1.910888,3,"1,2,3"


In [12]:
proactive_cities_index = (
    pd.concat(
        [
            selected_outliers["PV_R"]
            .sort_values("inc_ratio", ascending=False)
            .head(samples["PV_R"])[ftr_pref_muni],
            selected_outliers["PV_S"]
            .sort_values("inc_ratio", ascending=False)
            .head(samples["PV_S"])[ftr_pref_muni],
        ]
    )
    .drop_duplicates()
    .reset_index(drop=True)
    .apply(tuple, axis=1)
    .values
)
print(len(proactive_cities_index))
proactive_cities_index

16


array([('鹿児島県', '鹿屋市'), ('愛知県', '名古屋市'), ('静岡県', '浜松市'), ('佐賀県', '佐賀市'),
       ('宮崎県', '都城市'), ('愛知県', '一宮市'), ('群馬県', '伊勢崎市'), ('群馬県', '太田市'),
       ('宮崎県', '宮崎市'), ('群馬県', '前橋市'), ('岡山県', '津山市'), ('大分県', '杵築市'),
       ('山梨県', '北杜市'), ('茨城県', '神栖市'), ('栃木県', '佐野市'), ('岡山県', '岡山市')],
      dtype=object)

In [13]:
# so: selected ourliers
so_stats = []
for vars_dv in ["PV_R", "PV_S"]:
    df_temp = selected_outliers[vars_dv]
    cols = [vars_dv, f"{vars_dv}_shap", "inc_ratio", 'cluster_p', "cluster_a"]
    df_temp = (
        df_temp[
            df_temp[ftr_pref_muni].apply(tuple, axis=1).isin(proactive_cities_index)
        ]
        .set_index(ftr_pref_muni)[cols]
        .rename(columns={"inc_ratio": f"{vars_dv}_ir", "cluster_a": f"{vars_dv}_cluster_a", "cluster_p": f"{vars_dv}_cluster_p"})
    )
    so_stats.append(df_temp.copy())

so_rename_cols = {
    "pref_en": "Pref",
    "muni_en": "Muni",
    "PV_R": "$PV_{R}^{actual}$",
    "PV_R_shap": "$PV_{R}^{shap}$",
    "PV_R_ir": "$PV_{R}^{ir}$",
    "PV_R_cluster_a": "$PV_{R}^{cl_a}$",
    "PV_R_cluster_p": "$PV_{R}^{cl_p}$",
    "PV_S": "$PV_{S}^{actual}$",
    "PV_S_shap": "$PV_{S}^{shap}$",
    "PV_S_ir": "$PV_{S}^{ir}$",
	"PV_S_cluster_a": "$PV_{S}^{cl_a}$",
    "PV_S_cluster_p": "$PV_{S}^{cl_p}$",
}
so_stats = pd.concat(so_stats, axis=1).reset_index()
so_stats.insert(2, "muni_en", so_stats["muni"].map(japanadmin_muni_all_jp_to_en))
so_stats.insert(2, "pref_en", so_stats["pref"].map(prefecture_dict_jp_to_en))
so_stats["muni_en"] = so_stats["muni_en"].apply(
    lambda x: x.replace("-shi", "").replace("-machi", "")
)
so_stats = so_stats.sort_values(["PV_S_ir", "PV_R_ir"], ascending=False)
so_stats.iloc[:, 4:6] = so_stats.iloc[:, 4:6].applymap(lambda x: f"{x: .4f}")
so_stats.iloc[:, 9:11] = so_stats.iloc[:, 9:11].applymap(lambda x: f"{x: .4f}")


so_stats = so_stats.rename(columns=so_rename_cols)
df_so_stats = so_stats[so_rename_cols.values()]
df_so_stats

,Pref,Muni,$PV_{R}^{actual}$,$PV_{R}^{shap}$,$PV_{R}^{ir}$,$PV_{R}^{cl_a}$,$PV_{R}^{cl_p}$,$PV_{S}^{actual}$,$PV_{S}^{shap}$,$PV_{S}^{ir}$,$PV_{S}^{cl_a}$,$PV_{S}^{cl_p}$
14,Okayama,Tsuyama,0.1432,0.1824,0.785280,"2,3,4,5",5,0.4033,0.0848,4.755758,"0,1",4
10,Oita,Kitsuki,0.0470,0.0375,1.252054,"3,4,5,6,7",6,0.3280,0.0918,3.573143,"0,1,2,3",4
4,Miyazaki,Miyakonojo,0.3727,0.2259,1.649840,1,2,0.6508,0.1889,3.444649,0,3
9,Gunma,Maebashi,0.5188,0.3761,1.379290,"0,1",1,0.8597,0.2597,3.310258,-,0
6,Gunma,Isesaki,0.3781,0.2424,1.559923,1,2,0.6127,0.1906,3.214705,0,1
0,Kagoshima,Kanoya,0.1907,0.0772,2.471788,"2,3,4",6,0.6143,0.2193,2.800344,0,3
2,Shizuoka,Hamamatsu,1.2680,0.7143,1.775003,-,0,1.3935,0.4990,2.792660,-,0
11,Yamanashi,Hokuto,0.0935,0.0774,1.207414,"2,3,4,5,6",6,0.5253,0.1949,2.694981,0,3
1,Aichi,Nagoya,1.4090,0.7831,1.799222,-,0,0.5463,0.2182,2.503482,0,2
5,Aichi,Ichinomiya,0.5455,0.3449,1.581279,"0,1",2,0.2991,0.1565,1.910888,"1,2,3",3
